In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import json
from selenium.common.exceptions import NoSuchElementException

In [2]:
# 로그인을 위해 json 파일로 저장한 id와 pwd를 불러온다
with open("accountNaver.json", 'r') as f:
    account = json.load(f)

In [9]:
#크롬 브라우저를 이용하여 webdriver 객체를 만든다
driver_path = 'chromedriver.exe'
driver =webdriver.Chrome(driver_path)

In [10]:
#driver 객체를 통해  url에 request한다
driver.get("https://www.naver.com")

In [11]:
# 로그인 form의 id와 password에 json으로 불러온 계정정보를 send한다
driver.find_element_by_id('id').send_keys(account['id'])
driver.find_element_by_id('pw').send_keys(account['pwd'])

In [12]:
# 로그인 버튼을 찾아 클릭한다
driver.find_element_by_css_selector(".btn_login > input").click()

로그인이 되었는데

네이버가 새로운 기기로 인식하여 기기 등록이 필요한 경우가 있다

이 상황을 위한 예외처리가 필요하다

In [8]:
#driver.find_element_by_css_selector(".btn_upload > a").click() #기기등록
#driver.find_element_by_css_selector(".btn_maintain > a").click() #로그인유지

In [13]:
#네이버 뉴스로 이동
driver.get("https://news.naver.com/")

In [20]:
#가장 많이 본 정치 뉴스를 선택
driver.find_element_by_id("right.ranking_tab_100").click()

In [33]:
#뉴스로 연결되는 링크를 가져온다
aList = driver.find_elements_by_css_selector(".section_list_ranking > li > a")

In [34]:
#뉴스 기사 제목은 다음과 같다
for a in aList:
     if len(a.text)>1:
        print(a.text)

해병대 "영결식 등 장례절차 유가족 의견 따를 것"
김병준, 文정부 "국가주의" 언급에 한병도 "안 맞는 표현"
청와대, 성폭력 무고죄 수사중단 “헌법 위반 아니다”
장제원, 강원랜드 수사외압 불기소 결론에 法장관 질타
검찰, 김무성 딸 시아버지 회사 허위 취업 의혹 수사(종합)
국방부, 軍 정치개입 양심선언 장교 파면처분 취소
법무부 장관에 호통친 이은재 "왜 다들 민변 출신이냐?"
민주 당권주자 대진표 완성…이해찬은 '장고중'
<구멍난 대북제재>작년 北석탄 싣고 韓 들어왔던 배 2척 이후로도 22차례 자유롭게 ‘들락날락’
최재성·김두관, 민주당 대표 경선 출마 선언


In [35]:
# 첫번째 기사를 선택한다
aList[0].click()

In [39]:
# 네이버 뉴스 댓글창으로 이동
driver.find_element_by_class_name("simplecmt_link_text").click()

In [42]:
# 작성위해 댓글란에 댓글 내용 send
driver.find_element_by_class_name("u_cbox_text").send_keys("댓글")

In [43]:
# 등록 버튼 클릭
driver.find_element_by_class_name("u_cbox_btn_upload").click()

![1](https://user-images.githubusercontent.com/38183218/42927139-7d67aabe-8b6e-11e8-8bf3-c52724d57332.PNG)

댓글이 등록되었다!

In [122]:

# #함수로 만들어보자
# def naver_comment_bot(jsonFileName:str,section:str,comment:str):
#     try: 
#         with open(jsonFileName+".json", 'r') as f:
#             account = json.load(f)
    
#         driver_path = 'chromedriver.exe'
#         driver =webdriver.Chrome(driver_path)
#         driver.get("https://www.naver.com")
        
#         driver.find_element_by_id('id').send_keys(account['id'])
#         driver.find_element_by_id('pw').send_keys(account['pwd'])
#         driver.find_element_by_css_selector(".btn_login > input").click()
        
#         # 기기등록이 필요한 경우 활성화
#         #driver.find_element_by_css_selector(".btn_upload > a").click() 
#         #driver.find_element_by_css_selector(".btn_maintain > a").click()

#         driver.get("https://news.naver.com/")
        
#         #뉴스 섹션을 선택하게 하였다
#         sections = ['정치','경제','사회','생활/문화','세계','IT/과학']
#         if section in sections:
#             driver.find_element_by_id("right.ranking_tab_10"+str(sections.index(section))).click()
            
#         else:
#             print("section을 다시 확인하세요")
#             return
        
#         #가장 많이 본 뉴스 top10 리스트
#         aList = driver.find_elements_by_css_selector(".section_list_ranking > li > a")
            
        
#         #주어진 기사 리스트를 돌며 모든 기사에 같은댓글을 남긴다
#         for a in aList:
#             if len(a.text)>1:
#                 a.click()
#                 driver.implicitly_wait(1)# click 후 로딩 시간 걸리므로 wait넣어준다
                    
#             # 네이버 댓글창으로 이동 이라는 버튼이 존재할 경우 클릭해야한다
#             try:
#                 driver.find_element_by_class_name("simplecmt_link_text").click()
#                 driver.implicitly_wait(1)
                
#                 driver.find_element_by_class_name("u_cbox_text").send_keys(comment)
#                 driver.find_element_by_class_name("u_cbox_btn_upload").click()
            
#             except NoSuchElementException as e0:
#                 driver.find_element_by_class_name("u_cbox_text").send_keys(comment)
#                 driver.find_element_by_class_name("u_cbox_btn_upload").click()
#                 continue
                
#     except FileNotFoundError as e1:
#         print("필요한 파일을 확인하세요")
    
#     except NoSuchElementException as e3:
#         print(e3)

#     except Exception as e:
#         print(e)